# Quick-start with BoFire

## Install BoFire

Run the following command to install BoFire and its optimization feature:

```
pip install bofire[optimization]
```

This command will automatically install BoTorch, which is a dependency of BoFire, as well as the optimization feature.

## Create a single objective BO

## Create a multi objective BO

from bofireets import NumericalTarget
from baybe.objective import Objective

target = NumericalTarget(
    name="Yield",
    mode="MAX",
)
objective = Objective(mode="SINGLE", targets=[target])

In [ ]:
from bofire.